In [144]:
import os
import nltk
import spacy
import es_core_news_sm
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from spacy.lang.es.stop_words import STOP_WORDS
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\acepi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\acepi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [145]:
# Inicializar el lematizador de spaCy
nlp = spacy.load('es_core_news_sm')

# Definir el directorio donde están los archivos .txt
directorio = 'DiscursosOriginales/'  # Reemplaza esto con tu directorio

# Lista para almacenar los contenidos de los archivos
contenidos = []

# Leer los textos y combinarlos en un solo corpus
for filename in os.listdir(directorio):
    if filename.endswith('.txt'):
        with open(os.path.join(directorio, filename), 'r', encoding='utf-8') as file:
            contenido = file.read()
            print(f'Archivo {filename} leído')

            # Tokenizar y lematizar el contenido
            tokens = word_tokenize(contenido.lower())# Convertir a minúsculas
            tokens_sin_stopwords = [word for word in tokens if word not in STOP_WORDS]
            doc = nlp(' '.join(tokens_sin_stopwords))
            lemas = [token.lemma_ for token in doc]

            # Convertir los lemas nuevamente a texto
            corpus_lematizado = ' '.join(lemas)
            contenidos.append(corpus_lematizado)


Archivo 100020.txt leído
Archivo 100033.txt leído
Archivo 100172.txt leído
Archivo 100178.txt leído
Archivo 100227.txt leído
Archivo 100260.txt leído
Archivo 100296.txt leído
Archivo 100302.txt leído
Archivo 100385.txt leído
Archivo 100422.txt leído
Archivo 100454.txt leído
Archivo 100509.txt leído
Archivo 100547.txt leído
Archivo 100603.txt leído
Archivo 100990.txt leído
Archivo 101030.txt leído
Archivo 101041.txt leído
Archivo 101075.txt leído
Archivo 101103.txt leído
Archivo 101169.txt leído
Archivo 101255.txt leído
Archivo 101299.txt leído
Archivo 101310.txt leído
Archivo 101337.txt leído
Archivo 101383.txt leído
Archivo 101415.txt leído
Archivo 101452.txt leído
Archivo 101556.txt leído
Archivo 101614.txt leído
Archivo 101688.txt leído
Archivo 101766.txt leído
Archivo 101951.txt leído
Archivo 102005.txt leído
Archivo 102024.txt leído
Archivo 102066.txt leído
Archivo 102200.txt leído
Archivo 102228.txt leído
Archivo 102280.txt leído
Archivo 102351.txt leído
Archivo 102437.txt leído


In [146]:
print(contenidos[0])

: disfrutir sol invierno . gabriela mistral decir “ amar ciudad ” agregar belleza ciudad ennoblecer fealdad envilecer . importante ciudad hogar . conocer país cuidar ciudad , chile aprender . importante , ver cabl , cable necesario , empresa requerir cabl telecomunicación , telefonía fijo televisión instalar cable , cabl quedar desuso preocupo sacarlo . cable usado , estar ver lograr propósito : ciudad bonito , hermoso disfrutar él , ciudad seguro , cable desuso representar peligro . poner campaña “ chao cable ” , ¿ él acordar “ chao suegra ” , ¿ y reir ? suegrar . “ chao suegra ” pareja joven tener vivir padre suegro , tener oportunidad vivienda ir feliz , ir iniciar vida autonomía , independencia . “ chao cabl ” ciudad linda , bonitar disfrutar él , seguro . mes marzo año , logrado sacar 647 kilómetro cable desuso . distancia santiago temuco , distancia santiago vallenar . : meter llegar -ant terminar gobierno- sacar 2 mil kilómetro cable desuso . tarea . querer agradecer , lugar , t

In [170]:
model = Word2Vec(tokens, vector_size=500, window=5, min_count=1, sg=0)

In [194]:
vectores_por_texto = []
for texto_tokens in contenidos:
    vectores_por_token = [model.wv[token] for token in texto_tokens if token in model.wv]
    if vectores_por_token:
        # Promediar los vectores para obtener un vector representativo del texto
        vector_promedio = sum(vectores_por_token) / len(vectores_por_token)
        vectores_por_texto.append(vector_promedio)
    else:
        # Si no hay tokens en el modelo, agregar un vector de ceros
        vectores_por_texto.append([0] * model.vector_size)

In [195]:
type(vectores_por_texto[0])

numpy.ndarray

In [196]:
consulta = str(input("Ingrese su consulta: "))
tokens_consulta = word_tokenize(consulta.lower())
embedding_consulta = [model.wv[token] for token in tokens_consulta if token in model.wv]


In [197]:
embedding_consulta = embedding_consulta[0].reshape(1, -1)

In [198]:
similitudes = []
counter = 0
for vector_texto in vectores_por_texto:
    vector = vector.reshape(1, -1)
    similitud = cosine_similarity(embedding_consulta, vector)[0][0]
    similitudes.append((contenidos[counter], similitud))
    counter += 1

# Ordenar por similitud descendente
similitudes = sorted(similitudes, key=lambda x: x[1], reverse=True)

In [199]:
for texto, similitud in similitudes[:5]:  # Muestra los 5 textos más relevantes
    print(f'Similitud: {similitud}')
    print(texto)
    print('-' * 30)

Similitud: -0.041751813143491745
: disfrutir sol invierno . gabriela mistral decir “ amar ciudad ” agregar belleza ciudad ennoblecer fealdad envilecer . importante ciudad hogar . conocer país cuidar ciudad , chile aprender . importante , ver cabl , cable necesario , empresa requerir cabl telecomunicación , telefonía fijo televisión instalar cable , cabl quedar desuso preocupo sacarlo . cable usado , estar ver lograr propósito : ciudad bonito , hermoso disfrutar él , ciudad seguro , cable desuso representar peligro . poner campaña “ chao cable ” , ¿ él acordar “ chao suegra ” , ¿ y reir ? suegrar . “ chao suegra ” pareja joven tener vivir padre suegro , tener oportunidad vivienda ir feliz , ir iniciar vida autonomía , independencia . “ chao cabl ” ciudad linda , bonitar disfrutar él , seguro . mes marzo año , logrado sacar 647 kilómetro cable desuso . distancia santiago temuco , distancia santiago vallenar . : meter llegar -ant terminar gobierno- sacar 2 mil kilómetro cable desuso . tar